In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import utils
utils.set_pretty_prints()

In [2]:
#SDK deployment
import azureml

In [3]:
#1. Stabilirea comunicarii intre notebook -> Resource group-ul creat in portal.azure.com (HODS)
from azureml.core import Workspace
ws = Workspace.from_config(path='azureml/config.json')

In [4]:
#2. Definirea asseturilor necesare ce vor fi transferate catre portal.azure.com
from azureml.core.model import Model
model = Model.register(workspace=ws, 
                      model_path='models',
                      model_name='restocracy',
                      tags={'restocracy': 'demo'},
                      description='Model trained on restocracy reviews')

Registering model restocracy


In [5]:
#3. Inregistrarea environmentului de inferenta
from azureml.core.environment import Environment
hods_env = Environment.from_conda_specification(name='hods_env', file_path='azureml/config.yaml')
hods_env.register(workspace=ws)

{
    "name": "hods_env",
    "version": "2",
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
                "python=3.6.2",
                "tensorflow>=2.0.0",
                {
                    "pip": [
                        "azureml-defaults>=1.0.45"
                    ]
                }
            ],
            "name": "azureml_394b48ebc78f500e7797523b76f34ef3"
        }
    },
    "docker": {
        "enabled": false,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseDockerfile": null,
        "sharedVolumes": true,
        "shmSize": null,
        "arguments": [],
        "baseImageRegistry": {
            "address": null,
            "username": null,
            

In [14]:
#4. Define inference confing
from azureml.core.model import InferenceConfig
inf_env = Environment.get(workspace=ws, name='hods_env', version='2')
inference_config = InferenceConfig(entry_script='azureml/score.py', environment=inf_env)


In [11]:
#5. Deploy in ACI (Azure Container Instances)
from azureml.core.webservice import AciWebservice, Webservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service = Model.deploy(ws, 'aciservice1', [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)
print(service.state)

Running.....................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [8]:
inference_config

InferenceConfig(entry_script=azureml/score.py, runtime=None, conda_file=None, extra_docker_file_steps=None, source_directory=None, enable_gpu=None, base_image=None, base_image_registry=<azureml.core.container_registry.ContainerRegistry object at 0x7f818c1f9748>)

In [17]:
#6. Test the webservice
import requests
import json
headers = {'Content-Type': 'application/json'}
payload = "am fost la un restaurant cu muzica placuta si personal dragut. ciorba lor de legume era delicioasa. fructele lor de mare erau din alta lume. caviarul a fost delicios. am uitat sa mentionez ca restaurantul era pe Calea Dorobanti"
response = requests.post(service.scoring_uri, data=payload, headers=headers)
print(service.scoring_uri)
print(response.status_code)
print(response.elapsed)
print(response)
print(response.json())

http://6e3445cb-c25d-474f-a8a3-7c554bb04efc.westeurope.azurecontainer.io/score
200
0:00:00.299337
<Response [200]>
Expecting value: line 1 column 1 (char 0)


In [15]:
service.update(models=[model], inference_config=inference_config)
print(service.state)
print(service.get_logs)

Healthy
<bound method Webservice.get_logs of AciWebservice(workspace=Workspace.create(name='Lecture8', subscription_id='21862221-de06-45b9-b99b-94e0a3c69193', resource_group='HODS'), name=aciservice1, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://6e3445cb-c25d-474f-a8a3-7c554bb04efc.westeurope.azurecontainer.io/score, properties={})>


In [26]:
#6. Test the webservice
import requests
import json
headers = {'Content-Type': 'application/json'}
payload = json.dumps({"text": "am fost la un restaurant cu muzica placuta si personal dragut. ciorba lor de legume era delicioasa. fructele lor de mare erau din alta lume. caviarul a fost delicios. am uitat sa mentionez ca restaurantul era pe Calea Dorobanti"})
response = requests.post(service.scoring_uri, data=payload, headers=headers)
print(service.scoring_uri)
print(response.status_code)
print(response.elapsed)
print(response.json())

http://6e3445cb-c25d-474f-a8a3-7c554bb04efc.westeurope.azurecontainer.io/score
200
0:00:00.326545
[[84.26679229736328]]


In [19]:
service.scoring_uri

'http://6e3445cb-c25d-474f-a8a3-7c554bb04efc.westeurope.azurecontainer.io/score'

In [24]:
type(response.json())

str